# Dynamic Charging (Using `routes-info.xml` + `sumolib`)

In [ ]:
import os, sys
if 'SUMO_HOME' in os.environ:
    tools = os.path.join(os.environ['SUMO_HOME'], 'tools')
    sys.path.append(tools)
else:   
    sys.exit("please declare environment variable 'SUMO_HOME'")
    
import pprint, math, datetime
from time import strftime, gmtime
from lxml import etree
import matplotlib.pyplot as plt
import numpy as np
import sumolib

scenario = "../simulation/charlottenburg"

netfile = scenario + "/network/network.net.xml"

In [ ]:
net = sumolib.net.readNet(netfile)

osm_road_types = [
    "motorway",
    "trunk",
    "primary",
    "secondary",
    "tertiary",
    "residential",
    "living_street",
    "motorway_link",
    "motorway_link",
    "trunk_link",
    "primary_link",
    "secondary_link",
    "tertiary_link",
    "road",
    "unclassified"
]

emission_classes = [
    "PC_G_EU0",
    "PC_G_EU1",
    "PC_G_EU2",
    "PC_G_EU3",
    "PC_G_EU4",
    "PC_G_EU5",
    "PC_G_EU6",
    "PC_D_EU0",
    "PC_D_EU1",
    "PC_D_EU2",
    "PC_D_EU3",
    "PC_D_EU4",
    "PC_D_EU5",
    "PC_D_EU6",
    "PC_Alternative",
]

distances_per_road_type = {road_type: 0 for road_type in osm_road_types}
distances_per_emission_class = {emission_class: 0 for emission_class in emission_classes}

### Necessary Data

We need the following information about every trip:
* Which edges did a person drive on that were in one of the zones?
* What distance did a person drive on each of the edges?
* Which zone did each edge belong to?
* What type of egdes was it (e.g. residential or main street)?
* Which emission class did the vehicle have?

Sidenote: An edge may cross multiple zones

We aggregate the data using the following data structure:
```python
{
    trip_id: {
        emission_class: "HBEFA3/PC_G_EU4",
        edge_id: {
            edge_type: "residential",
            distance_{zone}: 12345
        }
    }
}
```

In [ ]:
def get_distances(file_path, snapshot_zones=False):
    distances_per_trip = {}
  
    for event, element in etree.iterparse(file_path, tag="timestep"):
        current_zone_timestep = element.attrib["zone-timestep"]

        for vehicle in element.getchildren():
            vid = vehicle.attrib["id"]
            speed = float(vehicle.attrib["speed"])
            v_timestep = vehicle.attrib["zone-timestep"]
            edge = vehicle.attrib["edge"]
            emission_class = vehicle.attrib["emission-class"]

            distances_per_edge = {}
            if vid in distances_per_trip:
                distances_per_edge = distances_per_trip[vid]

            if emission_class not in distances_per_edge:
                distances_per_edge["emission_class"] = emission_class
            
            zone = None

            polygons = []
            holes = []

            # Filter for snapshot and sort into polygons and holes
            for polygon in vehicle.getchildren():
                p_timestep = polygon.attrib["zone-timestep"]
                if snapshot_zones:
                    if p_timestep != v_timestep:
                        continue
                else:
                    if p_timestep != current_zone_timestep:
                        continue

                pid = polygon.attrib["id"]
                if pid.startswith("hole"):
                    holes.append(polygon)
                else:
                    polygons.append(polygon) 

            if len(polygons) == 0:
                continue
            
            # Find polygon with max zones number
            polygon_ids = list(map(lambda p: p.attrib["id"], polygons))
            polygon_zones = list(map(lambda pid: int(pid.split("_")[0].split("-")[0]), polygon_ids))
            max_zone = max(polygon_zones)
            max_index = polygon_zones.index(max_zone)
            max_polygon = polygons[max_index]
            max_polygon_id = polygon_ids[max_index]

            # Check if there is a hole whose ID contains the polygon ID
            hole_ids = list(map(lambda h: h.attrib["id"], holes))

            if any(max_polygon_id in hole_id for hole_id in hole_ids):
                # If yes -> Vehicle is inside the hole
                continue
            else:
                # If no -> Vehicle is inside the polygon
                zone = max_zone

            # Now we know the vehicle was not in a holes and
            # we know which zones this part of the edge (and the vehicle) was in

            edge_distance = {}
            if edge in distances_per_edge:
                edge_distance = distances_per_edge[edge]
            else:
                egde_type = net.getEdge(edge).getType()
                edge_distance["edge_type"] = egde_type.split(".")[1]


            distances_per_road_type[edge_distance["edge_type"]] += speed
            distances_per_emission_class[emission_class.split("/")[1]] += speed

            zone_distance_key = f"distance_{zone}"
            if zone_distance_key in edge_distance:
                edge_distance[zone_distance_key] += speed
            else:
                edge_distance[zone_distance_key] = speed

            distances_per_edge[edge] = edge_distance
            distances_per_trip[vid] = distances_per_edge

        element.clear()

    return distances_per_trip

In [ ]:
distances = get_distances(f"{scenario}/output/vehicle-zone-tracking.xml")
# distances = get_distances(f"test.xml", snapshot_zones=True)

# pprint.pprint(distances)
# pprint.pprint(distances_per_road_type)
pprint.pprint(distances_per_emission_class)

In [ ]:
fig, ax = plt.subplots(figsize=(25,10))

x_values = np.arange(0, len(distances_per_road_type.keys()))
ax.bar(x_values, distances_per_road_type.values(), align="center")
ax.set_xticks(x_values)
ax.set_xticklabels(distances_per_road_type.keys())
ax.tick_params(axis="both", labelsize=12)
ax.set_xlabel("OSM Road Type", fontsize=15, labelpad=30)
ax.set_ylabel("Total Distance", fontsize=15, labelpad=30)
ax.set_title("Total Distance per OSM Road Type", fontsize=30, pad=50)

In [ ]:
fig, ax = plt.subplots(figsize=(25,10))

x_values = np.arange(0, len(distances_per_emission_class.keys()))
ax.bar(x_values, distances_per_emission_class.values(), align="center")
ax.set_xticks(x_values)
ax.set_xticklabels(distances_per_emission_class.keys())
ax.tick_params(axis="both", labelsize=12)
ax.set_xlabel("Emission Type", fontsize=15, labelpad=30)
ax.set_ylabel("Total Distance", fontsize=15, labelpad=30)
ax.set_title("Total Distance per Emission Type", fontsize=30, pad=50)